# DermaLens - Dog Skin Lesion Classification Training
## CCS 248 Course Project

This notebook contains the complete training pipeline for a ResNet50 deep neural network trained from scratch on dog skin lesion images.

**Dataset**: 5,530 images across 6 disease classes  
**Model**: ResNet50 (25.5M parameters, from scratch)  
**Optimizer**: AdamW (lr=0.01, weight_decay=0.0001)  
**Training**: 143 epochs (best model at epoch 122)  
**Final Test Accuracy**: 86.61%

## 1. Import Required Libraries

Import all necessary libraries for deep learning, data processing, and visualization.

In [ ]:
# Core libraries
import os
import sys
import json
import time
from pathlib import Path
from typing import Dict, Tuple, Optional, List

# Data processing
import numpy as np
from PIL import Image

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# Machine learning utilities
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm.notebook import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
import yaml

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")

## 2. Load Configuration and Prepare Environment

Load training hyperparameters from config file and set up directories.

In [ ]:
# Load configuration from YAML file
with open('configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Display configuration
print("=" * 50)
print("TRAINING CONFIGURATION")
print("=" * 50)
print(f"Model: {config['model']['architecture']}")
print(f"Pretrained: {config['model']['pretrained']}")  # Should be False
print(f"Num Classes: {config['model']['num_classes']}")
print(f"Dropout Rate: {config['model']['dropout_rate']}")
print()
print(f"Learning Rate: {config['training']['learning_rate']}")
print(f"Weight Decay: {config['training']['weight_decay']}")
print(f"Batch Size: {config['training']['batch_size']}")
print(f"Num Epochs: {config['training']['num_epochs']}")
print(f"Early Stopping Patience: {config['training']['early_stopping_patience']}")
print("=" * 50)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

# Create directories
Path(config['training']['model_save_dir']).mkdir(parents=True, exist_ok=True)
Path(config['training']['log_dir']).mkdir(parents=True, exist_ok=True)
print(f"Checkpoint dir: {config['training']['model_save_dir']}")
print(f"Log dir: {config['training']['log_dir']}")

## 3. Define Custom Dataset Class

Create PyTorch Dataset for loading and preprocessing dog skin lesion images.

In [ ]:
class DogSkinLesionDataset(Dataset):
    """Custom Dataset for dog skin lesion images"""
    
    def __init__(self, root_dir: str, split: str, transform=None):
        """
        Args:
            root_dir: Root directory containing dataset
            split: 'train', 'valid', or 'test'
            transform: Optional transform to be applied on images
        """
        self.root_dir = Path(root_dir) / split
        self.transform = transform
        self.classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Load all image paths and labels
        self.samples = []
        for class_name in self.classes:
            class_dir = self.root_dir / class_name
            for img_path in class_dir.glob('*'):
                if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                    self.samples.append((str(img_path), self.class_to_idx[class_name]))
        
        print(f"{split.upper()} SET: {len(self.samples)} images across {len(self.classes)} classes")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Test dataset loading
print("\nTesting dataset loading...")
test_dataset = DogSkinLesionDataset(
    root_dir=config['dataset']['dataset_path'],
    split='train',
    transform=None
)
print(f"Classes: {test_dataset.classes}")
print(f"Sample image shape: {np.array(test_dataset[0][0]).shape}")

## 4. Define Data Augmentation and Create DataLoaders

Set up data augmentation for training and create DataLoaders for all splits.

In [ ]:
# Define transforms for training (with augmentation)
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(
        config['dataset']['image_size'],
        scale=(config['augmentation']['scale_min'], config['augmentation']['scale_max']),
        ratio=(config['augmentation']['aspect_ratio_min'], config['augmentation']['aspect_ratio_max'])
    ),
    transforms.RandomHorizontalFlip() if config['augmentation']['random_flip'] else transforms.Lambda(lambda x: x),
    transforms.RandomRotation(config['augmentation']['random_rotation']),
    transforms.ColorJitter(
        brightness=config['augmentation']['brightness'],
        contrast=config['augmentation']['contrast'],
        saturation=config['augmentation']['saturation'],
        hue=config['augmentation']['hue']
    ) if config['augmentation']['color_jitter'] else transforms.Lambda(lambda x: x),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define transforms for validation/test (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(config['dataset']['image_size']),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = DogSkinLesionDataset(
    root_dir=config['dataset']['dataset_path'],
    split=config['dataset']['train_split'],
    transform=train_transform
)

valid_dataset = DogSkinLesionDataset(
    root_dir=config['dataset']['dataset_path'],
    split=config['dataset']['valid_split'],
    transform=val_transform
)

test_dataset = DogSkinLesionDataset(
    root_dir=config['dataset']['dataset_path'],
    split=config['dataset']['test_split'],
    transform=val_transform
)

# Create dataloaders
batch_size = config['training']['batch_size']

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True if torch.cuda.is_available() else False
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"Train batches: {len(train_loader)}")
print(f"Valid batches: {len(valid_loader)}")
print(f"Test batches: {len(test_loader)}")

## 5. Define ResNet50 Model Architecture

Build ResNet50 from scratch with custom classification head.

In [ ]:
class DermaLensModel(nn.Module):
    """ResNet50-based model for dog skin lesion classification"""
    
    def __init__(self, num_classes=6, dropout_rate=0.3, pretrained=False):
        super().__init__()
        
        # Load ResNet50 backbone (from scratch or pretrained)
        self.backbone = models.resnet50(weights=None if not pretrained else models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Get number of features from backbone
        num_features = self.backbone.fc.in_features
        
        # Replace final layer with custom classification head
        self.backbone.fc = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)
    
    def count_parameters(self):
        """Count total and trainable parameters"""
        total = sum(p.numel() for p in self.parameters())
        trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return total, trainable

# Create model
model = DermaLensModel(
    num_classes=config['model']['num_classes'],
    dropout_rate=config['model']['dropout_rate'],
    pretrained=config['model']['pretrained']  # Should be False
).to(device)

# Display model info
total_params, trainable_params = model.count_parameters()
print("=" * 50)
print("MODEL ARCHITECTURE")
print("=" * 50)
print(f"Model: ResNet50")
print(f"Pretrained: {config['model']['pretrained']}")
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / 1024 / 1024:.1f} MB")
print("=" * 50)

## 6. Configure Optimizer, Loss Function, and Scheduler

Set up AdamW optimizer with weight decay, CrossEntropy loss, and ReduceLROnPlateau scheduler.

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer: AdamW (Adam with decoupled weight decay)
optimizer = optim.AdamW(
    model.parameters(),
    lr=config['training']['learning_rate'],
    weight_decay=config['training']['weight_decay']
)

# Learning rate scheduler: ReduceLROnPlateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',  # Maximize validation accuracy
    factor=0.5,  # Reduce LR by 50%
    patience=5,  # Wait 5 epochs before reducing
    verbose=True
)

print("=" * 50)
print("TRAINING SETUP")
print("=" * 50)
print(f"Optimizer: AdamW")
print(f"Learning Rate: {config['training']['learning_rate']}")
print(f"Weight Decay: {config['training']['weight_decay']}")
print(f"Loss Function: CrossEntropyLoss")
print(f"LR Scheduler: ReduceLROnPlateau (factor=0.5, patience=5)")
print("=" * 50)

## 7. Define Training and Validation Functions

Implement training loop and validation evaluation.

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    pbar = tqdm(dataloader, desc="Training")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc


def validate(model, dataloader, criterion, device):
    """Validate model"""
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Validating")
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    epoch_f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return epoch_loss, epoch_acc, epoch_f1

print("Training and validation functions defined.")

## 8. Train the Model

Run complete training loop with early stopping.

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'valid_loss': [],
    'valid_acc': [],
    'valid_f1': [],
    'learning_rate': []
}

# Early stopping variables
best_valid_acc = 0.0
best_epoch = 0
patience_counter = 0
num_epochs = config['training']['num_epochs']
early_stopping_patience = config['training']['early_stopping_patience']

print("\n" + "=" * 50)
print("STARTING TRAINING")
print("=" * 50)
print(f"Total Epochs: {num_epochs}")
print(f"Early Stopping Patience: {early_stopping_patience}")
print("=" * 50 + "\n")

start_time = time.time()

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 50)
    
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    valid_loss, valid_acc, valid_f1 = validate(model, valid_loader, criterion, device)
    
    # Update learning rate
    scheduler.step(valid_acc)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['valid_loss'].append(valid_loss)
    history['valid_acc'].append(valid_acc)
    history['valid_f1'].append(valid_f1)
    history['learning_rate'].append(current_lr)
    
    # Print epoch summary
    print(f"\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f} | Valid F1: {valid_f1:.4f}")
    print(f"Learning Rate: {current_lr:.6f}")
    
    # Save best model
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_epoch = epoch + 1
        patience_counter = 0
        
        # Save checkpoint
        checkpoint_path = Path(config['training']['model_save_dir']) / 'best_model.pth'
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'valid_acc': valid_acc,
            'valid_f1': valid_f1,
            'config': config
        }, checkpoint_path)
        print(f"✓ New best model saved! (Val Acc: {valid_acc:.4f})")
    else:
        patience_counter += 1
        print(f"No improvement for {patience_counter} epoch(s)")
    
    # Early stopping
    if patience_counter >= early_stopping_patience:
        print(f"\n⚠ Early stopping triggered after {epoch+1} epochs")
        print(f"Best validation accuracy: {best_valid_acc:.4f} at epoch {best_epoch}")
        break

training_time = time.time() - start_time
print("\n" + "=" * 50)
print("TRAINING COMPLETE")
print("=" * 50)
print(f"Total Training Time: {training_time/3600:.2f} hours")
print(f"Best Validation Accuracy: {best_valid_acc:.4f} at Epoch {best_epoch}")
print(f"Final Model: {checkpoint_path}")
print("=" * 50)

## 9. Save Training History

Save training metrics to JSON file.

In [ ]:
# Save training history
history_path = Path(config['training']['log_dir']) / 'training_history.json'
with open(history_path, 'w') as f:
    json.dump(history, f, indent=2)

print(f"Training history saved to: {history_path}")

## 10. Visualize Training Progress

Plot training and validation loss/accuracy curves.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0].plot(history['valid_loss'], label='Valid Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Plot accuracy
axes[1].plot(history['train_acc'], label='Train Accuracy', marker='o')
axes[1].plot(history['valid_acc'], label='Valid Accuracy', marker='s')
axes[1].axhline(y=best_valid_acc, color='r', linestyle='--', label=f'Best Val Acc: {best_valid_acc:.4f}')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plot_path = Path(config['training']['log_dir']) / 'training_history.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"Training plot saved to: {plot_path}")
plt.show()

## 11. Evaluate on Test Set

Load best model and evaluate on test set for final performance metrics.

In [ ]:
# Load best model
checkpoint = torch.load(checkpoint_path, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best model from epoch {checkpoint['epoch']}")

# Evaluate on test set
print("\nEvaluating on Test Set...")
test_loss, test_acc, test_f1 = validate(model, test_loader, criterion, device)

# Get predictions for all splits
def get_predictions(model, dataloader, device):
    """Get all predictions and labels"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    return np.array(all_preds), np.array(all_labels)

# Get predictions
train_preds, train_labels = get_predictions(model, train_loader, device)
valid_preds, valid_labels = get_predictions(model, valid_loader, device)
test_preds, test_labels = get_predictions(model, test_loader, device)

# Calculate metrics for all splits
results = {
    'train': {
        'accuracy': accuracy_score(train_labels, train_preds),
        'precision': precision_score(train_labels, train_preds, average='weighted'),
        'recall': recall_score(train_labels, train_preds, average='weighted'),
        'f1': f1_score(train_labels, train_preds, average='weighted')
    },
    'validation': {
        'accuracy': accuracy_score(valid_labels, valid_preds),
        'precision': precision_score(valid_labels, valid_preds, average='weighted'),
        'recall': recall_score(valid_labels, valid_preds, average='weighted'),
        'f1': f1_score(valid_labels, valid_preds, average='weighted')
    },
    'test': {
        'accuracy': accuracy_score(test_labels, test_preds),
        'precision': precision_score(test_labels, test_preds, average='weighted'),
        'recall': recall_score(test_labels, test_preds, average='weighted'),
        'f1': f1_score(test_labels, test_preds, average='weighted')
    }
}

# Print results
print("\n" + "=" * 50)
print("FINAL EVALUATION RESULTS")
print("=" * 50)
for split, metrics in results.items():
    print(f"\n{split.upper()} SET:")
    print(f"  Accuracy:  {metrics['accuracy']:.4f}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
print("=" * 50)

# Save results
results_path = Path(config['training']['log_dir']) / 'evaluation_results.json'
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)
print(f"\nResults saved to: {results_path}")

## 12. Generate Confusion Matrix

Visualize per-class prediction performance.

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(test_labels, test_preds)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=config['dataset']['class_names'],
            yticklabels=config['dataset']['class_names'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title(f'Confusion Matrix - Test Set\nOverall Accuracy: {results["test"]["accuracy"]:.4f}')
plt.tight_layout()

cm_path = Path(config['training']['log_dir']) / 'confusion_matrix.png'
plt.savefig(cm_path, dpi=150, bbox_inches='tight')
print(f"Confusion matrix saved to: {cm_path}")
plt.show()

# Per-class metrics
print("\nPER-CLASS PERFORMANCE (Test Set):")
print("-" * 70)
print(f"{'Class':<20} {'Precision':<12} {'Recall':<12} {'F1 Score':<12}")
print("-" * 70)

for i, class_name in enumerate(config['dataset']['class_names']):
    # Get samples for this class
    class_mask = test_labels == i
    class_preds = test_preds[class_mask]
    class_labels = test_labels[class_mask]
    
    if len(class_labels) > 0:
        prec = precision_score(class_labels, class_preds, labels=[i], average='micro')
        rec = recall_score(class_labels, class_preds, labels=[i], average='micro')
        f1 = f1_score(class_labels, class_preds, labels=[i], average='micro')
        print(f"{class_name:<20} {prec:<12.4f} {rec:<12.4f} {f1:<12.4f}")

print("-" * 70)

## 13. Make Sample Predictions

Test the trained model on sample images from the test set.

In [ ]:
# Get a batch of test images
images, labels = next(iter(test_loader))
images = images.to(device)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(images)
    probabilities = torch.softmax(outputs, dim=1)
    _, predictions = torch.max(outputs, 1)

# Denormalize images for visualization
mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

# Plot sample predictions
num_samples = min(8, len(images))
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for i in range(num_samples):
    # Denormalize image
    img = images[i].cpu() * std + mean
    img = torch.clamp(img, 0, 1)
    img = img.permute(1, 2, 0).numpy()
    
    # Plot
    axes[i].imshow(img)
    axes[i].axis('off')
    
    # Get prediction info
    true_label = config['dataset']['class_names'][labels[i]]
    pred_label = config['dataset']['class_names'][predictions[i]]
    confidence = probabilities[i][predictions[i]].item()
    
    # Color based on correctness
    color = 'green' if predictions[i] == labels[i] else 'red'
    axes[i].set_title(f"True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2%}", 
                     color=color, fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n✓ Displayed {num_samples} sample predictions")

## 14. Training Summary

Display comprehensive summary of the training process and final results.

In [ ]:
print("\n" + "=" * 70)
print(" " * 20 + "TRAINING SUMMARY")
print("=" * 70)
print(f"\n{'CONFIGURATION':-^70}")
print(f"Model:                ResNet50 (from scratch)")
print(f"Total Parameters:     {total_params:,}")
print(f"Pretrained:           {config['model']['pretrained']}")
print(f"Optimizer:            AdamW")
print(f"Learning Rate:        {config['training']['learning_rate']}")
print(f"Weight Decay:         {config['training']['weight_decay']}")
print(f"Batch Size:           {config['training']['batch_size']}")
print(f"Training Time:        {training_time/3600:.2f} hours")
print(f"Epochs Completed:     {len(history['train_loss'])}/{num_epochs}")
print(f"\n{'DATASET':-^70}")
print(f"Training Images:      {len(train_dataset):,}")
print(f"Validation Images:    {len(valid_dataset):,}")
print(f"Test Images:          {len(test_dataset):,}")
print(f"Total Images:         {len(train_dataset) + len(valid_dataset) + len(test_dataset):,}")
print(f"Number of Classes:    {config['model']['num_classes']}")
print(f"\n{'FINAL RESULTS':-^70}")
print(f"Best Validation Acc:  {best_valid_acc:.4f} (Epoch {best_epoch})")
print(f"Test Accuracy:        {results['test']['accuracy']:.4f}")
print(f"Test Precision:       {results['test']['precision']:.4f}")
print(f"Test Recall:          {results['test']['recall']:.4f}")
print(f"Test F1 Score:        {results['test']['f1']:.4f}")
print(f"\n{'GENERALIZATION':-^70}")
print(f"Train Accuracy:       {results['train']['accuracy']:.4f}")
print(f"Validation Accuracy:  {results['validation']['accuracy']:.4f}")
print(f"Test Accuracy:        {results['test']['accuracy']:.4f}")
gap = results['test']['accuracy'] - results['validation']['accuracy']
print(f"Test-Val Gap:         {gap:+.4f} {'(Excellent!)' if gap >= 0 else '(Overfitting)'}")
print(f"\n{'OUTPUT FILES':-^70}")
print(f"Best Model:           {checkpoint_path}")
print(f"Training History:     {history_path}")
print(f"Evaluation Results:   {results_path}")
print(f"Training Plot:        {plot_path}")
print(f"Confusion Matrix:     {cm_path}")
print("=" * 70)
print("\n✓ TRAINING COMPLETE - Model ready for deployment!")
print(f"✓ To use this model, load from: {checkpoint_path}")
print("=" * 70)